<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - Create an Empty DataFrame & RDD](https://sparkbyexamples.com/pyspark/pyspark-create-an-empty-dataframe/)

# 🗂️ PySpark - יצירת DataFrame ו-RDD ריקים

**תאריך:** 27 במרץ, 2024  
**זמן קריאה משוער:** 6 דקות

---

## 🎯 מבוא

במאמר זה, אסביר כיצד ליצור PySpark DataFrame/RDD ריק באופן ידני עם או בלי schema (שמות עמודות) בדרכים שונות.

להלן הסברתי את אחד מהתרחישים הרבים שבהם אנו צריכים ליצור DataFrame ריק.

### 📋 מתי נצטרך DataFrame ריק?

בזמן עבודה עם קבצים, לפעמים ייתכן שלא נקבל קובץ לעיבוד, אולם עדיין נצטרך ליצור DataFrame באופן ידני עם אותו schema שאנו מצפים לו.

אם לא ניצור עם אותו schema, הפעולות/טרנספורמציות שלנו (כמו `union()`) על DataFrame יכשלו כיוון שהן מתייחסות לעמודות שאולי לא קיימות.

**לטיפול במצבים דומים לאלה**, אנו תמיד צריכים ליצור DataFrame עם אותם שמות עמודות וטיפוסי נתונים, ללא קשר אם הקובץ קיים או ריק בזמן העיבוד.

## 1️⃣ יצירת RDD ריק ב-PySpark

צור RDD ריק באמצעות `emptyRDD()` של SparkContext, לדוגמה `spark.sparkContext.emptyRDD()`.

In [ ]:
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת RDD ריק
emptyRDD = spark.sparkContext.emptyRDD()
print(emptyRDD)

**הסבר:**

הקוד לעיל יוצר RDD ריק לחלוטין ללא partition.

### 🔄 חלופה: יצירת RDD ריק עם parallelize

לחלופין, ניתן גם לקבל RDD ריק באמצעות `spark.sparkContext.parallelize([])`.

In [ ]:
# יצירת RDD ריק באמצעות parallelize
rdd2 = spark.sparkContext.parallelize([])
print(rdd2)

**⚠️ שים לב:** אם תנסה לבצע פעולות על RDD ריק, תקבל שגיאה `ValueError("RDD is empty")`.

## 2️⃣ יצירת DataFrame ריק עם Schema (StructType)

כדי ליצור PySpark DataFrame ריק באופן ידני עם schema (שמות עמודות וסוגי נתונים), תחילה **צור schema באמצעות StructType ו-StructField**.

### 📐 שלב 1: יצירת Schema

In [ ]:
# יצירת Schema
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True)
])

**הסבר:**

- **StructType** - מגדיר את המבנה של ה-DataFrame
- **StructField** - מגדיר כל עמודה: שם, טיפוס נתונים, והאם ניתן להכיל NULL
- **StringType()** - טיפוס נתונים של מחרוזת
- **True** - מציין שהעמודה יכולה להכיל ערכי NULL

### 📊 שלב 2: יצירת DataFrame עם ה-Schema

In [ ]:
# יצירת DataFrame ריק מה-RDD הריק עם ה-schema
df = spark.createDataFrame(emptyRDD, schema)
df.printSchema()

**תוצאה צפויה:**

```
root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
```

הקוד לעיל מניב את ה-schema של ה-DataFrame הריק.

## 3️⃣ המרת RDD ריק ל-DataFrame

ניתן גם ליצור DataFrame ריק על ידי המרת RDD ריק ל-DataFrame באמצעות `toDF()`.

In [ ]:
# המרת RDD ריק ל-DataFrame
df1 = emptyRDD.toDF(schema)
df1.printSchema()

**הסבר:**

השיטה `toDF()` ממירה RDD ל-DataFrame עם ה-schema שצוין.

## 4️⃣ יצירת DataFrame ריק עם Schema באופן ישיר

עד כה כיסיתי יצירת DataFrame ריק מ-RDD, אך כאן ניצור אותו באופן ידני עם schema **וללא RDD**.

In [ ]:
# יצירת DataFrame ריק באופן ישיר
df2 = spark.createDataFrame([], schema)
df2.printSchema()

**הסבר:**

במקום להעביר RDD, אנו פשוט מעבירים רשימה ריקה `[]` יחד עם ה-schema, וזה יוצר DataFrame ריק עם המבנה המבוקש.

## 5️⃣ יצירת DataFrame ריק ללא Schema (ללא עמודות)

כדי ליצור DataFrame ריק ללא schema (ללא עמודות), פשוט צור schema ריק והשתמש בו בזמן יצירת PySpark DataFrame.

In [ ]:
# יצירת DataFrame ריק ללא schema (ללא עמודות)
df3 = spark.createDataFrame([], StructType([]))
df3.printSchema()

**תוצאה:**

```
#root
```

זה מדפיס schema ריק לחלוטין.

## 📌 סיכום

במדריך זה למדנו מספר דרכים ליצירת DataFrame ו-RDD ריקים ב-PySpark:

### ✅ שיטות ליצירת RDD ריק:
1. **emptyRDD()** - יצירת RDD ריק לחלוטין
2. **parallelize([])** - יצירת RDD ריק עם partitions

### ✅ שיטות ליצירת DataFrame ריק:
1. **עם Schema** - באמצעות StructType ו-StructField
2. **המרה מ-RDD** - באמצעות toDF()
3. **יצירה ישירה** - באמצעות createDataFrame([], schema)
4. **ללא Schema** - DataFrame ריק לחלוטין ללא עמודות

### 🎯 נקודות מפתח:
- שימוש ב-DataFrame ריק עם schema עוזר למנוע שגיאות בפעולות union ו-transformations
- חשוב להגדיר schema אחיד גם כאשר אין נתונים
- ניתן ליצור DataFrame ריק עם או בלי schema בהתאם לצורך

## 📚 מאמרים קשורים

- **PySpark Replace Empty Value With None/null on DataFrame**
- **Create a PySpark DataFrame from Multiple Lists**
- **PySpark Create RDD with Examples**
- **PySpark SparkContext Explained**
- **PySpark Replace Column Values in DataFrame**
- **PySpark Retrieve DataType & Column Names of DataFrame**
- **PySpark Count of Non null, nan Values in DataFrame**
- **PySpark RDD Actions with examples**

## 🏷️ תגיות (Tags)

`EMPTY DATAFRAME` `EMPTYRDD`

## 🔗 מקורות ומידע נוסף

**מקור המדריך:**
- SparkByExamples.com

**תיעוד רשמי:**
- [Apache Spark Documentation](https://spark.apache.org/docs/latest/)
- [PySpark SQL Documentation](https://spark.apache.org/docs/latest/api/python/)

---

**💻 בהצלחה בלימוד PySpark!**

**✨ Happy Learning !!**